In [1]:
import struct
import os

original_files = os.listdir("Archivos")
original_dxr_files = [f for f in original_files if (f.endswith(".DXR") or f.endswith(".CXT"))]
print(original_dxr_files)


In [2]:
# ===== READING THE HEADER =====

print("===== Reading the header =====")

# opening the target file...
file = open("Archivos/PUEBLO.DXR", "rb")

# reading the header...
file_header = file.read(4).decode('ascii') # always XFIR
print("Header: " + file_header)

# reading the file size
file_size, = struct.unpack("<I", file.read(4))
print("File size: " + str(file_size) + " Bytes")

# reading the file type
file_type = file.read(4).decode('ascii')
print("File type: " + file_type) # always 39VM

In [3]:
# ===== READING THE INITIAL MAP =====

print("===== Reading the initial map =====")

# reading imap's header
imap_header = file.read(4).decode('ascii')
print("imap header: " + imap_header) # always pami

# reading imap's size
imap_size, = struct.unpack("<I", file.read(4))
print("imap size: " + str(imap_size) + " Bytes")

# reading the number of mmap's
mmap_count, = struct.unpack("<I", file.read(4))
print("mmap count: " + str(mmap_count)) # hopefully always 1

# reading mmap's offset
mmap_offset, = struct.unpack("<I", file.read(4))
print("mmap offset: " + str(mmap_offset))

# reading mmap's version
mmap_version, = struct.unpack("<I", file.read(4))
print("mmap version: " + str(mmap_version)) # hopefully useless

In [4]:
# ===== READING THE MEMORY MAP =====

print("===== Reading the memory map =====")

# pointing to the memory map thanks to previous data
file.seek(mmap_offset, 0)

# reading mmap's header
mmap_header = file.read(4).decode('ascii')
print("mmap header: " + mmap_header)

# reading mmap's size
mmap_size, = struct.unpack("<I", file.read(4))
print("mmap size: " + str(mmap_size) + " Bytes")

# reading mmap's properties size
mmap_prop_size, = struct.unpack("<H", file.read(2))
print("mmap properties size: " + str(mmap_prop_size) + " Bytes")

# reading the size of each resource entry in the mmap
mmap_resource_entry_size, = struct.unpack("<H", file.read(2))
print("mmap resource entry size: " + str(mmap_resource_entry_size) + " Bytes")

# reading the maximum resources that can fit in the mmap
mmap_max_resource_count, = struct.unpack("<I", file.read(4))
print("mmap max resource count: " + str(mmap_max_resource_count))

# reading the actual resources in the mmap
mmap_resource_count, = struct.unpack("<I", file.read(4))
print("mmap resource count: " + str(mmap_resource_count))

# reading the last junk resource id
mmap_last_junk, = struct.unpack("<I", file.read(4))
print("mmap last junk resource id: " + str(mmap_last_junk)) # hopefully useless

# reading the old mmap resource id
mmap_old_mmap_id, = struct.unpack("<I", file.read(4))
print("mmap old mmap resource id: " + str(mmap_old_mmap_id)) # hopefully meaningless

# reading into the unknown
mmap_unk1, = struct.unpack("<I", file.read(4))
print("mmap unknown: " + str(mmap_unk1)) # hopefully useless

# loading resource entries
print("Loading resource table of contents...")
resources = []
for i in range(0, mmap_resource_count):
    tag = file.read(4).decode('ascii')
    size, = struct.unpack("<I", file.read(4))
    offset, = struct.unpack("<i", file.read(4))
    flags, = struct.unpack("H", file.read(2))
    unk1, = struct.unpack("H", file.read(2))
    next_free_resource_id, = struct.unpack("<I", file.read(4))
    resources.append((i,offset,size,tag))
print("Resource table of contents loaded.")

In [5]:
# ===== READING THE KEY* =====

print("===== Reading the KEY* =====")

key = resources[3] # it is known to always be the third.

# pointing to the KEY* thanks to the resource table of contents
file.seek(key[1], 0)

# reading KEY*'s header
keys_header = file.read(4).decode('ascii')
print("KEY* header: " + keys_header) # always *YEK

# reading KEY*'s size
keys_size, = struct.unpack("<I", file.read(4))
print("KEY* size: " + str(keys_size) + " Bytes")

# reading KEY*'s properties size
keys_prop_size, = struct.unpack("<H", file.read(2))
print("KEY* properties size: " + str(keys_prop_size) + " Bytes")

# reading the size of each key entry in KEY*
keys_entry_size, = struct.unpack("<H", file.read(2))
print("KEY* entry size: " + str(keys_entry_size) + " Bytes")

# reading the maximum keys that can fit in KEY*
keys_max, = struct.unpack("<I", file.read(4))
print("mmap max resource count: " + str(keys_max))

# reading the actual keys in KEY*
keys_count, = struct.unpack("<I", file.read(4))
print("mmap resource count: " + str(keys_count))

# loading keys
print("Loading the keys...")
owned = {}
for i in range(0, keys_count):
    index_owned, = struct.unpack("<I", file.read(4))
    owners_index, = struct.unpack("<I", file.read(4))
    owned_chunk_id = file.read(4).decode('ascii')
    if owners_index not in owned:
        owned[owners_index] = []
    owned[owners_index].append((index_owned, owned_chunk_id))
print("Keys loaded.")

In [6]:
# ===== READING THE CAS*'s =====

print("===== Reading the CAS*'s =====")

# Just brute-force search them in the resource table of contents
print("Searching for CAS* resources...")
cass = [x for x in resources if x[3] == "*SAC"]
print(str(len(cass)) + " CAS*'s found.")

cast_owned = {}

for i in range(0, len(cass)):
    cast_owned[i] = []
    
    # pointing to each CAS* thanks to the resource table of contents
    file.seek(cass[i][1], 0)
    
    # reading CAS*'s header
    cas_header = file.read(4).decode('ascii')
    print("CAS* " + str(i) + " header: " + cas_header) # always *SAC
    
    # reading CAS*'s size
    cas_size, = struct.unpack("<I", file.read(4))
    print("CAS* " + str(i) + " size: " + str(cas_size) + " Bytes")
    
    # computing the total cast members of each cast
    cast_members = int(cas_size/4)
    print("CAS* " + str(i) + " cast members: " + str(cast_members))
    
    for j in range (0, cast_members):
        index, = struct.unpack(">I", file.read(4)) # Beware the endianness!
        cast_owned[i].append(index)

===== Reading the CAS*'s =====
Searching for CAS* resources...
5 CAS*'s found.
CAS* 0 header: *SAC
CAS* 0 size: 704 Bytes
CAS* 0 cast members: 176
CAS* 1 header: *SAC
CAS* 1 size: 96 Bytes
CAS* 1 cast members: 24
CAS* 2 header: *SAC
CAS* 2 size: 112 Bytes
CAS* 2 cast members: 28
CAS* 3 header: *SAC
CAS* 3 size: 96 Bytes
CAS* 3 cast members: 24
CAS* 4 header: *SAC
CAS* 4 size: 500 Bytes
CAS* 4 cast members: 125


In [7]:
# ===== READING THE CASTMEMBERS =====

print("===== Reading the CastMembers =====")

bitmaps = []
sounds = []
other = []

for i in range(0, len(cass)):
    print("Processing CAS* " + str(i) + "...")
    for j in range(0, len(cast_owned[i])):
        castmember_toc = resources[cast_owned[i][j]]
        if castmember_toc[3] != 'tSAC':
            continue
        # pointing to each castmember thanks to the resource toc
        file.seek(castmember_toc[1], 0)
        file.read(8) # just skip the header and the size (we already know)
        
        # reading the castmember's type
        castmember_type, = struct.unpack(">I", file.read(4))
        if castmember_type == 1: #Bitmap
            # print(str(j) + ": " + str(castmember_toc) + " is a bitmap")
            if castmember_toc[0] in owned:
                bitmaps.append(castmember_toc[0])
                # print(owned[castmember_toc[0]])
        elif castmember_type == 6: #Sound
            # print(str(j) + ": " + str(castmember_toc) + " is a sound")
            if castmember_toc[0] in owned:
                sounds.append(castmember_toc[0])
                # print(owned[castmember_toc[0]])
        else :
            other.append(castmember_toc[0])
            # print("I don't care about " + str(castmember_toc))

print("Done.")
        

===== Reading the CastMembers =====
Processing CAS* 0...
Processing CAS* 1...
Processing CAS* 2...
Processing CAS* 3...
Processing CAS* 4...
Done.


In [8]:
# ===== READING BITMAPS =====

bmp_titles = []

# some little icons may give problems because they are in system mac
# maybe just discard them all?

for i in range(0, len(bitmaps)):
    
    # pointing to the castmember
    file.seek(resources[bitmaps[i]][1],0)
    file.read(12) # skipping header, size and type (we already know)
    prop1_size, = struct.unpack(">I", file.read(4)) # 26 + 9 + name or simply 26
    prop2_size, = struct.unpack(">I", file.read(4)) # always 28?
    if prop1_size == 26:
        file.read(26)
    else:
        file.read(35) # skip useless stuff
    if prop1_size-35 >= 0:
        bmp_titles.append(str(file.read(prop1_size-35))) #don't decode because of the tildes
    else:
        bmp_titles.append("untitled"+str(i))
    
    file.read(2)
    p1,p2,q1,q2 = struct.unpack(">HHHH", file.read(8))
    file.seek(-8,1)
    pp1,pp2,qq1,qq2 = struct.unpack("<HHHH", file.read(8))
    file.read(1)
    file.read(7)
    t1,t2=struct.unpack(">HH", file.read(4))
    file.read(1)
    depth, = struct.unpack(">c", file.read(1))
    depth = ord(depth)
    palette, = struct.unpack(">i", file.read(4))
    if (depth == 0):
        palette = -1
        depth = 1
        #print(bmp_titles[i] + " " + str(p1) + " " + str(p2) + " " + str(q1) + " " +str(q2) + " " + " " +str(depth) + " " +str(palette))
    w = q2 - p2
    h = q1 - p1
    
    # saving raw bitmaps and properties safely
    print(str(i)+" "+bmp_titles[i])
    print(w)
    print(h)
    bitmap_prop_file = open("IMAGENES/"+"untitled"+str(i)+".PROP","w")
    bitmap_prop_file.write(str(w)+"\n")
    bitmap_prop_file.write(str(h)+"\n")
    bitmap_prop_file.write(str(palette)+"\n")
    bitmap_prop_file.write(str(depth)+"\n")
    bitmap_raw_file = open("IMAGENES/"+"untitled"+str(i)+".BITD","wb")
    file.seek(resources[owned[bitmaps[i]][0][0]][1],0)
    file.read(8)
    bitmap_raw_file.write(file.read(resources[owned[bitmaps[i]][0][0]][2]))
    bitmap_prop_file.close()
    bitmap_raw_file.close()
    #print(resources[owned[bitmaps[i]][0][0]])
    

0 b'DIAG 1'
65
165
1 b'DIAG 2'
54
166
2 b'DIAG 3'
53
175
3 b'DIAG 4'
53
170
4 b'DIAG 5'
69
162
5 b'DIAG 6'
66
171
6 b'DIAG 7'
58
172
7 b'DIAG 8'
56
165
8 b'PERFIL 1'
78
162
9 b'PERFIL 2'
72
167
10 b'PERFIL 3'
71
168
11 b'PERFIL 4'
73
166
12 b'PERFIL 5'
76
160
13 b'PERFIL 6'
72
166
14 b'PERFIL 7'
69
164
15 b'PERFIL 8'
72
165
16 b'REPOSO'
70
167
17 b'DIGES 1'
71
158
18 b'DIGES 2'
66
167
19 b'DIGES 3'
64
170
20 b'DIGES 4'
69
165
21 b'DIGES 5'
69
163
22 b'DIGES 6'
64
165
23 b'DIGES 7'
64
166
24 b'DIGES 8'
64
162
25 b'1DOWN'
104
167
26 b'1UP'
97
167
27 b'2DOWN'
109
167
28 b'2UP'
102
167
29 b'REPOSO2'
70
167
30 b'MASCARA ICONO'
55
57
31 b'SALTO 1'
46
55
32 b'REPOSO'
70
167
33 b'ESPALDA 1'
54
171
34 b'ESPALDA 2'
57
164
35 b'ESPALDA 3'
61
173
36 b'ESPALDA 4'
56
170
37 b'ESPALDA 5'
53
165
38 b'ESPALDA 6'
56
169
39 b'ESPALDA 7'
62
172
40 b'ESPALDA 8'
55
171
41 b'REPOSOarriba'
70
167
42 untitled42
16
15
43 untitled43
16
15
44 untitled44
15
16
45 untitled45
15
16
46 untitled46
15
16
47 untitled47


In [9]:
# ===== READING SOUNDS =====

for i in range(0, len(sounds)):
    snd_file = open('test' + str(i) + '.snd_', 'wb')
    sample_file = open('test' + str(i) + '.sample', 'wb')
    snds = [x for x in owned[sounds[i]] if x[1] == ' dns']

    file.seek(resources[snds[0][0]][1],0)
    file.read(8) # snd_
    snd_file.write(file.read(4))
    binary_snd_header_size = file.read(4)
    snd_header_size, = struct.unpack("<I", binary_snd_header_size)
    snd_file.write(binary_snd_header_size)
    snd_file.write(file.read(snd_header_size))
        
    file.read(4)
    sample_size, = struct.unpack("<I", file.read(4))
    sample_file.write(file.read(sample_size))
        
    snd_file.close()
    sample_file.close()

In [10]:
file.close()

In [11]:
# INSPIRED BY https://github.com/System25/drxtract/blob/master/snd2wav

import wave
import os

raw_file = open('test0.sample', 'rb')
file_size = os.path.getsize('test0.sample')
print(file_size)

bps = 16

num_frames = file_size # file size (bytes)
wavef = wave.open("test0.wav",'w')
nsamples = int(num_frames)
wavef.setnchannels(1)
wavef.setsampwidth(int(bps/8)) #bps/8 
wavef.setframerate(22050)
if bps == 16:
    for f in range(0, int(nsamples/2)):
        #print("ok")
        sample =  int(struct.unpack(">H", raw_file.read(2))[0])
        data = struct.pack('<H', sample)
        wavef.writeframesraw( data )
else:
    wavef.writeframesraw( raw_file.read(file_size) )
wavef.writeframes(b'')
wavef.close()
os.system("ffmpeg -y -i test0.wav -acodec libmp3lame test0.mp3")

9470


0

In [12]:
# RLE DECOMPRESSION ALGORITHM MOSTLY TAKEN FROM https://github.com/System25/drxtract/blob/master/bitd2bmp

import logging

def save_16bit_bmp(bmp_width, bmp_height, file, fdata, name):
    
    bmp_bpp = 16
    
    # THERE IS A PADDING PROBLEM!!!
    padding = (4 - (bmp_width * int(bmp_bpp/8)) % 4) % 4
    # print("padding " + str(padding))
    
    file.write('BM'.encode('ascii'))
    hsize = 40
    values = (((bmp_width*2)+padding)*bmp_height+hsize+14, # The size of the BMP file in bytes
              0, # Reserved
              0, # Reserved
              (hsize+14) # Data offset
             )
    s = struct.Struct('<ihhi')
    packed_data = s.pack(*values)
    file.write(packed_data)

    # Write BITMAPINFOHEADER
    values = (hsize, # the size of this header (hsize bytes)
              bmp_width, # the bitmap width in pixels (signed integer)
              bmp_height, # the bitmap height in pixels (signed integer)
              1, # the number of color planes (must be 1)
              bmp_bpp, # the number of bits per pixel, which is the color depth of the image. Typical values are 1, 4, 8, 16, 24 and 32.
              0, # the compression method being used (BI_BITFIELDS)
              0, # the image size. This is the size of the raw bitmap data; a dummy 0 can be given for BI_RGB bitmaps.
              0, # the horizontal resolution of the image. (pixel per meter, signed integer)
              0, # the vertical resolution of the image. (pixel per meter, signed integer)
              0, # the number of colors in the color palette, or 0 to default to 2n
              0, # the number of important colors used, or 0 when every color is important; generally ignored
             )
    s = struct.Struct('<iiihhIIIIII')
    packed_data = s.pack(*values)
    file.write(packed_data)

    # get the pixel information
    # RLE encoded bytes are:
    #   - RLE encoded lower byte
    #   - RLE encoded upper byte

    w = bmp_width*2
    h = bmp_height
    
    castData = [200 for x in range((w+padding)*bmp_height)]
    x = 0
    y = bmp_height - 1
    idx = 0
    
    compress = False
    
    if (len(fdata) == bmp_width*bmp_height*2):
        print(name + " PROBABLY UNCOMPRESSED")
        print(padding)
        for i in range(h-1,-1,-1):
            for j in range(0,w,2):
                castData[i*(w+padding)+j+1] = struct.unpack("B", fdata[idx:idx+1])[0]
                idx += 1
                castData[i*(w+padding)+j] = struct.unpack("B", fdata[idx:idx+1])[0]
                idx += 1
    else:
        compress = True
        vueltas = 0
        while (idx < len(fdata)) and (y>=0) and True:
            #logging.debug("HERE:"+str(type(fdata)))
            val = struct.unpack("B", fdata[idx:idx+1])[0]
            if (val & 0x80) != 0:
                # RLE encoded
                run_length = 257 - val
                run_value = struct.unpack("B", fdata[idx+1:idx+2])[0]
                idx = idx + 2

                # Jump to next byte when necessary
                if ((x + run_length) > bmp_width) and (x < bmp_width):
                    x = bmp_width

                # Jump to next row when necessary
                if ((x + run_length) > w):
                    x = 0
                    y -= 1

                for i in range(0, run_length):
                    castData[y*(w+padding) + x] = run_value
                    x += 1

            else:
                # Not RLE encoded
                run_length = val + 1
                idx = idx + 1

                # Jump to next byte when necessary
                if ((x + run_length) > bmp_width) and (x < bmp_width):
                    x = bmp_width

                # Jump to next row when necessary
                if ((x + run_length) > w):
                    x = 0
                    y -= 1

                for i in range(0, run_length):
                    castData[y*(w+padding) + x] = struct.unpack("B", fdata[idx:idx+1])[0]
                    idx = idx + 1
                    x += 1
                    if x >= w:
                        x = 0
                        y -= 1
            vueltas +=1
            if vueltas < 0:
                break
    
    #print(y)
    #print(x)
    
    
    
    if not ((y == 0 and x == w) or (y == -1 and x == 0)) and compress:
        logging.warning(name + " Probably not properly generated")
    
    
    # Order lower and upper bytes
    castDatamix = [200 for x in range(((bmp_width*2)+padding)*bmp_height)]
    
    if compress:
        w2 = bmp_width*2 + padding
        w1 = bmp_width
        w0 = 0
        for y in range(0, bmp_height):
            yw1 = y*w1
            yw2 = y*w2
            for x in range(0, bmp_width):
                castDatamix[yw2 + x*2 + 0] = castData[yw2 + w1 + x]  # Upper
                castDatamix[yw2 + x*2 + 1] = castData[yw2 + w0 + x]  # Lower
    else:
        # print(castData)
        castDatamix=castData
    
    # Write the pixel information
    file.write(struct.pack("B"*(((bmp_width)*2+padding)*bmp_height), *castDatamix))
    file.close()

In [13]:
import os

images = os.listdir('IMAGENES')

for f in images:
    if f.endswith(".BITD"):
        name = os.path.splitext(f)[0]
        if (name + ".PROP" in images) and True:
            prop_file = open("IMAGENES/"+name + ".PROP", 'r')
            w = prop_file.readline()
            h = prop_file.readline()
            p = prop_file.readline()
            b = prop_file.readline()
            #print(b)
            #print(name)

            bitd_sample_file = open("IMAGENES/"+f,'rb')
            buffer = bitd_sample_file.read()
            out = open("IMAGENES/"+name+".bmp",'wb')

            if int(b) == 16:
                save_16bit_bmp(int(w),int(h),out,buffer,name)
                out.close()
                bitd_sample_file.close()
            else:
                print(name + " still unsupported")
        else:
            print("no properties file")


untitled42 still unsupported
untitled50 still unsupported
untitled107 still unsupported
untitled55 still unsupported
untitled45 still unsupported
untitled44 still unsupported
untitled47 still unsupported
untitled52 still unsupported
untitled51 still unsupported
untitled48 still unsupported
untitled46 still unsupported
untitled316 still unsupported
untitled53 still unsupported
untitled85 still unsupported
untitled49 still unsupported
untitled54 still unsupported
untitled43 still unsupported
untitled128 still unsupported
untitled168 PROBABLEMENTE SIN COMPRESIÓN
2
